In [1]:
import pandas as pd
import advertools as adv
import yagmail
from datetime import date
import requests
import configparser

In [14]:
# Read configs

config = configparser.ConfigParser()
config.read('config.ini')

auth_params = {
    'app_key': config['twitter']['API_Key'],
    'app_secret': config['twitter']['API_Key_Secret'],
    'oauth_token': config['twitter']['Access_Tocken'],
    'oauth_token_secret': config['twitter']['Access_Tocken_Secret'],
}

In [15]:
tw = adv.twitter.search(q='#confirmed', count = 100, tweet_mode='extended',lang='en')

2022-11-01 16:11:11,624 | INFO | twitter.py:246 | wrapper | search | Requesting: count=100, max_id=None, q=#confirmed, tweet_mode=extended, lang=en


TwythonAuthError: Twitter API returned a 400 (Bad Request), Bad Authentication data.

In [6]:
def send_alert(emailmessage, method):
    if method == 1:
        body = emailmessage
        yag = yagmail.SMTP("nrainer001@gmail.com")
        yag.send(
            to="nree254@outlook.com",
            subject="Twitter Alert",
            contents=body)
        print("Email sent")
    if method == 0:
        resp = requests.post('https://textbelt.com/text', {
        'phone': '+254746217838',
        'message': twitterhandle + " tweeted out: " + tweet,
        'key': '',
        })
        print(resp.json())

In [7]:
today = str(date.today())
adv.twitter.set_auth_params(**auth_params)

In [8]:
alerts_list = {"searchliaison":"core|update|algorithm","googlesearchc":"search/sconsole"} 
df1 = pd.DataFrame(columns = ['TwitterHandle', 'Message'])

In [9]:
for keys,values in alerts_list.items():
  df = adv.twitter.get_user_timeline(screen_name=keys,tweet_mode="extended")

2022-11-01 15:45:44,823 | INFO | twitter.py:246 | wrapper | get_user_timeline | Requesting: count=200, max_id=None, screen_name=searchliaison, tweet_mode=extended


TwythonAuthError: Twitter API returned a 400 (Bad Request), Bad Authentication data.

In [16]:
df.head()

,tweet_created_at,tweet_id,tweet_id_str,tweet_full_text,tweet_truncated,tweet_display_text_range,tweet_entities,tweet_entities_mentions,tweet_entities_hashtags,tweet_entities_urls,...,user_profile_use_background_image,user_has_extended_profile,user_default_profile,user_default_profile_image,user_following,user_follow_request_sent,user_notifications,user_translator_type,user_withheld_in_countries,tweet_source_url
0,2022-10-31 16:57:05+00:00,1587126523945013248,1587126523945013248,If you want your product web pages to standout...,False,"[0, 117]","{'hashtags': [], 'symbols': [], 'user_mentions...",,,https://goo.gle/3zuXKbH,...,True,False,False,False,False,False,False,none,[],https://www.sprinklr.com
1,2022-10-26 13:30:01+00:00,1585262476618350601,1585262476618350601,Hashtags in URL's... does Google ignore them? ...,False,"[0, 203]","{'hashtags': [{'text': 'JavaScript', 'indices'...",@JohnMu,"#JavaScript, #AskGoogleBot",http://goo.gle/3VUOcQV,...,True,False,False,False,False,False,False,none,[],https://www.sprinklr.com
2,2022-10-21 13:19:20+00:00,1583447847474630656,1583447847474630656,"This month's questions came from @amitsh053, @...",False,"[0, 178]","{'hashtags': [], 'symbols': [], 'user_mentions...","@amitsh053, @Bruroar, @dakswang, @HebaSaidSEO,...",,,...,True,False,False,False,False,False,False,none,[],https://mobile.twitter.com
3,2022-10-21 13:19:19+00:00,1583447844803211267,1583447844803211267,Want to submit your question for the next vers...,False,"[0, 182]","{'hashtags': [], 'symbols': [], 'user_mentions...",,,https://forms.gle/Ba685cc5xjJbR8r28,...,True,False,False,False,False,False,False,none,[],https://mobile.twitter.com
4,2022-10-21 13:19:18+00:00,1583447840616894464,1583447840616894464,Thanks to everyone who submitted questions for...,False,"[0, 199]","{'hashtags': [], 'symbols': [], 'user_mentions...",,,https://youtu.be/FtJhOrE0lxo,...,True,False,False,False,False,False,False,none,[],https://mobile.twitter.com


In [17]:
df['tweet_created_at'] = df['tweet_created_at'].astype('string') 
df = df[df['tweet_full_text'].str.contains(values,regex=True) & df['tweet_created_at'].str.contains(today)]

In [18]:
if len(df.index) > 0:
   df1 = df1.append({'TwitterHandle' : keys, 'Message' : df['tweet_full_text']}, ignore_index = True)

In [19]:
getlist = dict(zip(df1['TwitterHandle'].tolist(),df1['Message'].tolist()))
emailmessage = ""

In [20]:
if len(df1.index) > 0:
    for key, value in getlist.items():
        emailmessage += key + ": " + value + "\n\n"
        method = 0 
        send_alert(emailmessage,method)